# Проект по SQL

## Шаг 1. Подключение к базе данных

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import create_engine


# устанавка параметров
db_config = {'user': ###, # имя пользователя
             'pwd': ###, # пароль
             'host': ###,
             'port': ###, # порт подключения
             'db': ###} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Функция для запрсов

In [2]:
# функция для запросов
def q(query):
    return display(pd.io.sql.read_sql(query, con = engine))

## Шаг 2. Обзор данных

In [3]:
# таблица books
q('''SELECT * 
     FROM books 
     LIMIT 5;''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# таблица authors
q('''SELECT * 
     FROM authors 
     LIMIT 5;''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# таблица publishers
q('''SELECT * 
     FROM publishers 
     LIMIT 5;''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# таблица ratings
q('''SELECT * 
     FROM ratings
     LIMIT 5;''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# таблица reviews
q('''SELECT * 
     FROM reviews
     LIMIT 5;''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Шаг 3. Анализ данных

### Количество книг, вышедших после 1 января 2000 года

In [8]:
q('''SELECT COUNT(title)
     FROM books
     WHERE CAST(publication_date AS date) >= '2000-01-01';''')

,count
0,821


После 1 января 2000 года вышла 821 книга. 

Для каждой книги подсчитано количество обзоров и средняя оценка, это может помочь в отборе наиболее популярных книг:
- количество обзоров варьируется от 0 до 1120;
- средняя оценка варьируется от NaN до 5.0.

### Определение издательства, которое выпустило наибольшее число книг толще 50 страниц — так будут исключены из анализа брошюры

In [9]:
q('''SELECT publisher
     FROM
         (SELECT p.publisher,
                 COUNT(b.book_id) AS count_book
         FROM publishers AS p
         INNER JOIN books AS b ON p.publisher_id = b.publisher_id
         WHERE b.num_pages > 50
         GROUP BY p.publisher
         ORDER BY count_book DESC) AS q_1
      LIMIT 1;''')

,publisher
0,Penguin Books


Издательством, выпустившим наибольшее число книг толще 50 страниц, является `Penguin Books`(42 книги).

### Определение автора с самой высокой средней оценкой книг — учитены только книги с 50 и более оценками

In [10]:
q(''' SELECT author
      FROM 
          (SELECT author,
                  avg_rating
           FROM
               (SELECT a.author,
                       COUNT(rt.book_id) AS count_book,
                       AVG(rt.rating) AS avg_rating
                FROM authors AS a
                INNER JOIN books AS b ON a.author_id = b.author_id
                INNER JOIN ratings AS rt ON b.book_id = rt.book_id
                GROUP BY a.author
                ORDER BY count_book DESC) AS q_1
            WHERE count_book >= 50
            ORDER BY avg_rating DESC) AS q_2
        LIMIT 1;''')

,author
0,Diana Gabaldon


Автором с самой высокой средней оценкой книг, из книг получивших 50 и более оценок, является `Diana Gabaldon`(оценка 4.3).

### Посчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
q('''SELECT ROUND(AVG(avg_review)) AS avg_review
     FROM
         (SELECT avg_review
          FROM
              (SELECT rv.username,
                      AVG(rv.review_id) AS avg_review,
                      COUNT(rt.rating) AS count_rating
               FROM reviews AS rv
               INNER JOIN ratings AS rt ON rv.book_id = rt.book_id
               GROUP BY rv.username
               ORDER BY count_rating DESC) AS q_1
          WHERE count_rating > 50
          ORDER BY avg_review DESC) AS q_2;''')

,avg_review
0,1423.0


Среднее количество обзоров от пользователей, поставивших 50 и более оценок, составляет 1423.

## Шаг 4. Вывод

- После 1 января 2000 года вышла 821 книга.

- Для каждой книги подсчитано количество обзоров и средняя оценка, это может помочь в отборе наиболее популярных книг:
    - количество обзоров варьируется от 0 до 1120;
    - средняя оценка варьируется от NaN до 5.0.

- Издательством, выпустившим наибольшее число книг толще 50 страниц, является Penguin Books(42 книги).

- Автором с самой высокой средней оценкой книг, из книг получивших 50 и более оценок, является Diana Gabaldon(оценка 4.3).

- Среднее количество обзоров от пользователей, поставивших 50 и более оценок, составляет 1423.